In [1]:
import pandas as pd
import csv

from analyzer_clean import Flow, _FLOW_FIELDS

In [2]:
def get_volumetric_features(df, stats):
    stats['num_entries'] = len(df)
    
    # num => len(); count => unique.len()
    stats['count_dst_port'] = len( df['dst_port'].unique() )
    stats['count_src_port'] = len( df['src_port'].unique() )
    stats['count_dst_ip'] = len( df['dst_ip'].unique() )
    stats['count_src_ip'] = len( df['src_ip'].unique() )
    
    # number of packets (entries) in each state
    df_state_count = df.groupby('state')['dst_tx'].count()
    for state in ['closed', 'established', 'reset', 'new', 'closing', 'connecting']:
        if state in df_state_count:
            stats['num_state_'+state] = df_state_count[state]
    
    return stats

In [3]:
# TODO move func to extract features here
def get_distribution_features(df, stats, key):
    if key=='conn':
        gp1 = df.groupby(['dst_ip', 'src_ip'])
        stats['count_conns'] = len(gp1)
        # dst port per conn
        _dist = gp1['dst_port'].count()
        for k,v in describe_distribution(_dist).items():
            stats['dst_port_per_conn_'+k] = v
    elif key=='flow':
        gp2 = df.groupby(['dst_ip', 'dst_port', 'src_ip', 'src_port', 'ip_protocol'])
        stats['count_flows'] = len(gp2)
        # mean flows per conn
        stats['num_flows_per_conn_mean'] = stats['count_flows']/stats['count_conns']    
        ## entries per flow
        #num_conns_per_flow_dist = gp2['bytes_up'].count()
        #num_flow_per_conn_dist = gp[['src_port', 'dst_port', 'ip_protocol']].count()
    elif key=='dst_ip':
        gp3 = df.groupby(['dst_ip'])
        # dst_port by dst_ip
        _dist = gp3['dst_port'].count()
        for k,v in describe_distribution(_dist).items():
            stats['dst_port_per_dst_ip_'+k] = v
        # src_ip by dst_ip
        _dist = gp3['src_ip'].count()
        for k,v in describe_distribution(_dist).items():
            stats['src_ip_per_dst_ip_'+k] = v
    elif key=='src_ip':
        gp4 = df.groupby(['src_ip'])
        # dst_port by src_ip
        _dist = gp4['dst_port'].count()
        for k,v in describe_distribution(_dist).items():
            stats['dst_port_per_src_ip_'+k] = v
        # dst_ip by src_ip
        _dist = gp4['dst_ip'].count()
        for k,v in describe_distribution(_dist).items():
            stats['dst_ip_per_src_ip_'+k] = v
    elif key=='dst_port':
        gp5 = df.groupby(['dst_port'])
        # dst_ip by dst_port
        _dist = gp5['dst_ip'].count()
        for k,v in describe_distribution(_dist).items():
            stats['dst_ip_per_dst_port_'+k] = v
        # src_ip by dst_port
        _dist = gp5['src_ip'].count()
        for k,v in describe_distribution(_dist).items():
            stats['src_ip_per_dst_port_'+k] = v
    else:
        print("key??", key)
        
    return stats

In [4]:
import scipy.stats

def entropy(ds_data):
    p_data = ds_data.value_counts()           # counts occurrence of each value
    entropy = scipy.stats.entropy(p_data)  # get entropy from counts
    return entropy

def get_entropy(df, stats):
    cols = ['dst_ip', 'dst_port', 'src_ip', 'src_port', 'ip_protocol', 'state']
    df_ent = df[cols].apply(entropy)
    stats.update(df_ent.to_dict())
    return stats

In [27]:
def process(flow_list):
    bad_flow_list = []
    df = pd.DataFrame(flow_list)
    stats = {}
    
    ## VOLUMETRIC FEATURES ##stats = get_volumetric_features(df, stats)
    #stats = get_volumetric_features(df, stats)
    
    # DISTRIBUTION FEATURES
    ## (2) by conn => unique (dst_ip, src_ip)
    #stats = get_distribution_features(df, stats, 'conn')
    #    
    ## (3) by flow => (dst_ip, dst_port, src_ip, src_port, protocol)
    #stats = get_distribution_features(df, stats, 'flow')
    #
    ## (4) by dst_ip
    #stats = get_distribution_features(df, stats, 'dst_ip')
    #
    ## (5) by src_ip
    #stats = get_distribution_features(df, stats, 'src_ip')
    #
    ## (6) by dst_port
    #stats = get_distribution_features(df, stats, 'dst_port')
    
    # ENTROPY for packet headers
    stats = get_entropy(df, stats)
    
    return bad_flow_list

In [15]:
df = pd.DataFrame( DATA[0] )

In [ ]:
process

In [5]:
from collections import defaultdict
import numpy as np

DATA = []
FLOW_LIST = []
window_size = 100  # running window to calc entropy
ent_stats = defaultdict(list)
flow_counter = 0

with open("data.csv", "r") as csvfile:
    # pass input data stream as open("data.csv", "r") to csv.reader for testing
    # read and process line by line don't read into list
    fin = csv.reader(csvfile)
    for e in fin:
        flow = Flow.from_csv(e)
        FLOW_LIST.append(flow)
        
        if len(FLOW_LIST) >= window_size:
            # TODO
            df = pd.DataFrame(FLOW_LIST)
            cols = ['dst_ip', 'dst_port', 'src_ip', 'src_port', 'ip_protocol', 'state']
            df_ent = df[cols].apply(entropy)
            for k,v in df_ent.to_dict().items():
                ent_stats[k].append(v)
            ent_stats['flow_counter'].append(flow_counter)
            del FLOW_LIST[0]
        flow_counter += 1

KeyboardInterrupt: 

In [ ]:
df_ent = pd.DataFrame(ent_stats)